# Examen de la práctica 2 de APR, grupo 4CO21, 10 de enero de 2024
# Turno 1: de 19h a 19.45h

## El conjunto de datos caltech-101

Caltech-101 consta de imágenes de objetos de 101 clases, más una clase "cajón de sastre". Cada imagen está etiquetada con un único objeto. Cada clase contiene entre 40 y 800 imágenes aproximadamente, con alrededor de 9.000 imágenes en total. Las imágenes son de tamaños variables, de anchos y altos típicamente entre 200 y 300 píxeles. A continuación se carga este conjunto de datos mediante la librería tensorflow_datasets y se divide en una parte para entrenamiento (train_data), otra para validación (val_data) y otra para test (test_data).

In [26]:
import tensorflow as tf
import tensorflow_datasets as tfds
train_data, val_data, test_data = tfds.load('caltech101', split=['train[:80%]', 'train[80%:]', 'test'], as_supervised=True)
train_size=len(train_data)

## Ejercicio

Como hicimos en las sesiones 3 y 4 de la práctica, haz un fine-tuning (ajuste fino) de una red Keras pre-entrenada con ImageNet que clasifique el test de caltech-101 con la máxima precisión posible, preferiblemente superior al 80%.

In [27]:
print(train_size)

2448


In [28]:
from keras.applications.efficientnet import preprocess_input

img_size = (299, 299)
num_classes = 102

def preprocess(image, label):
    image = tf.image.resize(image, img_size)
    image = tf.cast(image, tf.float32)
    image = preprocess_input(image)
    label = tf.one_hot(label, num_classes)
    return image, label

In [29]:
train_data = train_data.map(preprocess)
val_data = val_data.map(preprocess)
test_data = test_data.map(preprocess)

In [30]:
from keras.applications.inception_v3 import InceptionV3
from keras.applications.efficientnet import EfficientNetB0

model = EfficientNetB0(input_shape=img_size + (3,),include_top=False, weights='imagenet')

In [31]:
from keras.layers import GlobalAveragePooling2D, Dense, Dropout
from keras.models import Model

for layer in model.layers:
    layer.trainable = False

x = GlobalAveragePooling2D()(model.output)
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(num_classes, activation='softmax')(x)

model = Model(inputs=model.input, outputs=output)

In [32]:
from keras.optimizers import Adam

opt=Adam(learning_rate=0.001)
model.compile(loss='categorical_crossentropy',
            optimizer=opt,
            metrics=['accuracy'])

In [33]:
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.models import load_model

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=0.00001)
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_accuracy', save_best_only=True, verbose=1)

epochs=10
batch_size=32
train_data= train_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_data = val_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)

history = model.fit(train_data,
                    epochs=epochs,
                    verbose=1,
                    validation_data=val_data,
                    callbacks=[reduce_lr,checkpoint])

Epoch 1/10
77/77 [==============================] - ETA: 0s - loss: 2.0975 - accuracy: 0.5764
Epoch 1: val_accuracy improved from -inf to 0.87255, saving model to best_model.h5
77/77 [==============================] - 25s 170ms/step - loss: 2.0975 - accuracy: 0.5764 - val_loss: 0.4615 - val_accuracy: 0.8725 - lr: 0.0010
Epoch 2/10
77/77 [==============================] - ETA: 0s - loss: 0.3425 - accuracy: 0.9048
Epoch 2: val_accuracy improved from 0.87255 to 0.90850, saving model to best_model.h5
77/77 [==============================] - 10s 133ms/step - loss: 0.3425 - accuracy: 0.9048 - val_loss: 0.3554 - val_accuracy: 0.9085 - lr: 0.0010
Epoch 3/10
77/77 [==============================] - ETA: 0s - loss: 0.1896 - accuracy: 0.9473
Epoch 3: val_accuracy did not improve from 0.90850
77/77 [==============================] - 10s 125ms/step - loss: 0.1896 - accuracy: 0.9473 - val_loss: 0.3538 - val_accuracy: 0.9085 - lr: 0.0010
Epoch 4/10
77/77 [==============================] - ETA: 0s - l

In [34]:
model = load_model('best_model.h5')
test_dataset_batched = test_data.batch(batch_size).prefetch(tf.data.AUTOTUNE)
score = model.evaluate(test_dataset_batched, verbose=0)
print(f'Test loss: {score[0]*100:.2f}')
print(f'Test accuracy: {score[1]*100:.2f}')

Test loss: 41.52
Test accuracy: 88.21
